# Train Custom openWakeWord Model

Windows-compatible version of the openWakeWord training notebook.

## Prerequisites

### Setup the Jupyter kernel (one-time)

```bash
uv add ipykernel --dev
uv run python -m ipykernel install --user --name voice-gateway
```

### Run in VS Code

1. Open this notebook in VS Code
2. Click **Select Kernel** (top right) → **voice-gateway**
3. Run the cells!

## 1. Install Dependencies

In [ ]:
# Set to True for NVIDIA GPU support (requires CUDA installed)
USE_GPU = True

In [ ]:
import subprocess
import sys

# Install PyTorch with CUDA support from PyTorch's index
if USE_GPU:
    print("Installing PyTorch with CUDA support...")
    subprocess.run([
        "uv", "pip", "install",
        "torch>=2.0,<2.6", "torchaudio",
        "--index-url", "https://download.pytorch.org/whl/cu124"
    ], check=True)
else:
    print("Installing PyTorch (CPU only)...")
    subprocess.run(["uv", "pip", "install", "torch>=2.0,<2.6", "torchaudio"], check=True)

# Verify GPU is available
import torch
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU detected - will use CPU")

In [ ]:
# Core dependencies (Windows-compatible)
deps = [
    "piper-tts>=1.2.0",
    "numpy",
    "scipy",
    "tqdm",
    "datasets==2.14.6",
    "pyyaml",
    "onnxruntime",
    "onnx",
    "pronouncing",
    "deep-phonemizer",
    "mutagen",
    "torchinfo",
    "torchmetrics",
    "speechbrain==0.5.14",
    "requests",
    "ipywidgets",
    "soundfile",
]

# Install dependencies
failed = []
for dep in deps:
    result = subprocess.run(["uv", "pip", "install", dep], capture_output=True, text=True)
    if result.returncode != 0:
        failed.append(dep)
        print(f"Failed: {dep}")
    else:
        print(f"Installed: {dep}")

if failed:
    print(f"\nWarning: Failed to install: {failed}")
else:
    print("\nAll dependencies installed successfully!")

## 2. Setup

In [ ]:
import os
import subprocess
import wave
import json
import random
from pathlib import Path
import requests
from tqdm.auto import tqdm
import numpy as np
import torch
import torchaudio
import scipy.signal
from numpy.lib.format import open_memmap

SCRIPT_DIR = Path(".").resolve()
print(f"Working directory: {SCRIPT_DIR}")

In [ ]:
# Download piper voice model (ONNX format - works on Windows)
PIPER_MODELS_DIR = SCRIPT_DIR / "piper_models"
PIPER_MODELS_DIR.mkdir(exist_ok=True)

MODEL_NAME = "en_US-libritts_r-medium"
PIPER_MODEL_PATH = PIPER_MODELS_DIR / f"{MODEL_NAME}.onnx"
PIPER_CONFIG_PATH = PIPER_MODELS_DIR / f"{MODEL_NAME}.onnx.json"

base_url = "https://huggingface.co/rhasspy/piper-voices/resolve/main/en/en_US/libritts_r/medium"

for filepath, filename in [(PIPER_MODEL_PATH, f"{MODEL_NAME}.onnx"), (PIPER_CONFIG_PATH, f"{MODEL_NAME}.onnx.json")]:
    if not filepath.exists():
        print(f"Downloading {filename}...")
        url = f"{base_url}/{filename}"
        response = requests.get(url, stream=True)
        response.raise_for_status()
        total = int(response.headers.get("content-length", 0))
        with open(filepath, "wb") as f:
            with tqdm(total=total, unit="B", unit_scale=True, desc=filename) as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    pbar.update(len(chunk))
    else:
        print(f"{filename} already exists")

In [ ]:
# Clone openwakeword
OWW_DIR = SCRIPT_DIR / "openwakeword"

if not OWW_DIR.exists():
    print("Cloning openwakeword...")
    subprocess.run(["git", "clone", "https://github.com/dscripka/openwakeword"], cwd=SCRIPT_DIR, check=True)
else:
    print("openwakeword already exists")

In [ ]:
# Download openwakeword embedding models
OWW_MODELS_DIR = OWW_DIR / "openwakeword" / "resources" / "models"
OWW_MODELS_DIR.mkdir(parents=True, exist_ok=True)

model_urls = {
    "embedding_model.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx",
    "melspectrogram.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx",
}

for filename, url in model_urls.items():
    filepath = OWW_MODELS_DIR / filename
    if not filepath.exists():
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    else:
        print(f"{filename} already exists")

## 3. Download Augmentation Data

Download room impulse responses (RIRs) and background noise for data augmentation.
This significantly improves model quality by simulating real-world acoustic conditions.

In [ ]:
import zipfile
import tarfile

AUGMENT_DIR = SCRIPT_DIR / "augmentation_data"
AUGMENT_DIR.mkdir(exist_ok=True)

RIR_DIR = AUGMENT_DIR / "rir"
NOISE_DIR = AUGMENT_DIR / "noise"

In [ ]:
# Download MIT Acoustics RIR dataset (small, ~50MB)
RIR_DIR.mkdir(exist_ok=True)

# Using the OpenSLR RIR dataset (simulated room impulse responses)
RIR_URL = "https://www.openslr.org/resources/28/rirs_noises.zip"
RIR_ZIP = AUGMENT_DIR / "rirs_noises.zip"

if not (RIR_DIR / "RIRS_NOISES").exists():
    if not RIR_ZIP.exists():
        print("Downloading RIR dataset (~300MB)...")
        response = requests.get(RIR_URL, stream=True)
        response.raise_for_status()
        total = int(response.headers.get("content-length", 0))
        with open(RIR_ZIP, "wb") as f:
            with tqdm(total=total, unit="B", unit_scale=True, desc="RIR dataset") as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    pbar.update(len(chunk))
    
    print("Extracting RIR dataset...")
    with zipfile.ZipFile(RIR_ZIP, 'r') as z:
        z.extractall(RIR_DIR)
    print("RIR dataset ready")
else:
    print("RIR dataset already exists")

In [ ]:
# Download MUSAN noise dataset (background noise, music, babble)
NOISE_DIR.mkdir(exist_ok=True)

MUSAN_URL = "https://www.openslr.org/resources/17/musan.tar.gz"
MUSAN_TAR = AUGMENT_DIR / "musan.tar.gz"

if not (NOISE_DIR / "musan").exists():
    if not MUSAN_TAR.exists():
        print("Downloading MUSAN noise dataset (~11GB, this will take a while)...")
        response = requests.get(MUSAN_URL, stream=True)
        response.raise_for_status()
        total = int(response.headers.get("content-length", 0))
        with open(MUSAN_TAR, "wb") as f:
            with tqdm(total=total, unit="B", unit_scale=True, desc="MUSAN dataset") as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                    pbar.update(len(chunk))
    
    print("Extracting MUSAN dataset (this may take a few minutes)...")
    with tarfile.open(MUSAN_TAR, 'r:gz') as tar:
        tar.extractall(NOISE_DIR)
    print("MUSAN dataset ready")
else:
    print("MUSAN dataset already exists")

In [ ]:
# Collect all RIR and noise file paths
rir_paths = []
for ext in ["*.wav", "*.flac"]:
    rir_paths.extend(list((RIR_DIR / "RIRS_NOISES" / "simulated_rirs").rglob(ext)))
    rir_paths.extend(list((RIR_DIR / "RIRS_NOISES" / "real_rirs_isotropic_noises").rglob(ext)))

noise_paths = []
musan_dir = NOISE_DIR / "musan"
if musan_dir.exists():
    # Include noise and music, skip speech (we want non-speech background)
    noise_paths.extend(list((musan_dir / "noise").rglob("*.wav")))
    noise_paths.extend(list((musan_dir / "music").rglob("*.wav")))

print(f"Found {len(rir_paths)} RIR files")
print(f"Found {len(noise_paths)} noise/music files")

## 4. Test Wake Word Pronunciation

Before training, verify the TTS pronounces your wake word correctly.

**Tips:**
- If pronunciation is wrong, spell it phonetically with underscores: `"hey_seer_e"` for "hey siri"
- Spell out numbers: `"two"` not `"2"`
- Avoid punctuation except `?` and `!`

In [ ]:
# Configure your wake word here!
TARGET_WORD = "Seraphina"  # Change this to your desired wake word

In [ ]:
from piper import PiperVoice
from piper.config import SynthesisConfig
from IPython.display import Audio, display

# Load the voice model
voice = PiperVoice.load(str(PIPER_MODEL_PATH), str(PIPER_CONFIG_PATH))

# Get number of speakers
with open(PIPER_CONFIG_PATH, encoding="utf-8") as f:
    voice_config = json.load(f)
num_speakers = voice_config.get("num_speakers", 1)
print(f"Voice model loaded with {num_speakers} speakers")

In [ ]:
def generate_sample(text: str, output_path: Path, speaker_id: int = 0, 
                    length_scale: float = 1.0, noise_scale: float = 0.667, 
                    noise_w_scale: float = 0.8):
    """Generate a single audio sample using piper-tts."""
    config = SynthesisConfig(
        speaker_id=speaker_id,
        length_scale=length_scale,
        noise_scale=noise_scale,
        noise_w_scale=noise_w_scale,
    )
    
    with wave.open(str(output_path), "wb") as wav_file:
        voice.synthesize_wav(text, wav_file, syn_config=config)

# Test pronunciation
test_path = SCRIPT_DIR / "test_generation.wav"
generate_sample(TARGET_WORD, test_path, speaker_id=0, length_scale=1.1)
display(Audio(str(test_path), autoplay=True))
print(f"\nTest audio saved to: {test_path}")

In [ ]:
# Try different speakers to hear variations
print("Generating samples with different speakers...")
for speaker_id in range(min(5, num_speakers)):
    sample_path = SCRIPT_DIR / f"test_speaker_{speaker_id}.wav"
    generate_sample(TARGET_WORD, sample_path, speaker_id=speaker_id)
    print(f"Speaker {speaker_id}:")
    display(Audio(str(sample_path), autoplay=False))

## 5. Generate Training Samples

Generate diverse audio samples of the wake word using different speakers and synthesis parameters.

In [ ]:
# Training sample configuration
N_SAMPLES_TRAIN = 2000  # Training samples (more = better, but slower)
N_SAMPLES_TEST = 200    # Test/validation samples
AUGMENTATION_ROUNDS = 3  # How many augmented versions of each sample to create

# Output directories matching openwakeword's expected structure
MODEL_OUTPUT_NAME = TARGET_WORD.replace(" ", "_")
OUTPUT_DIR = SCRIPT_DIR / "my_custom_model"
MODEL_DIR = OUTPUT_DIR / MODEL_OUTPUT_NAME

POSITIVE_TRAIN_DIR = MODEL_DIR / "positive_train"
POSITIVE_TEST_DIR = MODEL_DIR / "positive_test"
NEGATIVE_TRAIN_DIR = MODEL_DIR / "negative_train"
NEGATIVE_TEST_DIR = MODEL_DIR / "negative_test"

for d in [POSITIVE_TRAIN_DIR, POSITIVE_TEST_DIR, NEGATIVE_TRAIN_DIR, NEGATIVE_TEST_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {MODEL_DIR}")

In [ ]:
def generate_samples_batch(text: str, output_dir: Path, n_samples: int, desc: str = "Generating"):
    """Generate diverse training samples."""
    # Parameter ranges for variation
    length_scales = [0.8, 0.9, 1.0, 1.1, 1.2]
    noise_scales = [0.5, 0.667, 0.8]
    noise_w_scales = [0.6, 0.8, 1.0]
    
    generated = 0
    for i in tqdm(range(n_samples), desc=desc):
        output_path = output_dir / f"sample_{i:05d}.wav"
        
        if output_path.exists():
            generated += 1
            continue
        
        speaker_id = random.randint(0, num_speakers - 1)
        length_scale = random.choice(length_scales)
        noise_scale = random.choice(noise_scales)
        noise_w_scale = random.choice(noise_w_scales)
        
        try:
            generate_sample(
                text, output_path,
                speaker_id=speaker_id,
                length_scale=length_scale,
                noise_scale=noise_scale,
                noise_w_scale=noise_w_scale,
            )
            generated += 1
        except Exception as e:
            print(f"Failed sample {i}: {e}")
    
    return generated

# Generate positive samples
print(f"\nGenerating {N_SAMPLES_TRAIN} positive training samples...")
n = generate_samples_batch(TARGET_WORD, POSITIVE_TRAIN_DIR, N_SAMPLES_TRAIN, "Positive train")
print(f"Generated {n} training samples")

print(f"\nGenerating {N_SAMPLES_TEST} positive test samples...")
n = generate_samples_batch(TARGET_WORD, POSITIVE_TEST_DIR, N_SAMPLES_TEST, "Positive test")
print(f"Generated {n} test samples")

In [ ]:
# Generate adversarial negative samples (similar-sounding words)
# Inlined from openwakeword.data to avoid audiomentations dependency
import itertools
import re
import pronouncing

def phoneme_replacement(input_chars, max_replace, replace_char='(.){1,3}'):
    """Generate regex patterns with phonemes replaced."""
    results = []
    chars = list(input_chars)
    for r in range(1, max_replace + 1):
        comb = itertools.combinations(range(len(chars)), r)
        for indices in comb:
            chars_copy = chars.copy()
            for i in indices:
                chars_copy[i] = replace_char
            results.append(' '.join(chars_copy))
    return results

def generate_adversarial_texts(input_text: str, N: int, include_partial_phrase: float = 0, include_input_words: float = 0):
    """Generate adversarial words based on phoneme overlap."""
    vowel_phones = ["AA", "AE", "AH", "AO", "AW", "AX", "AXR", "AY", "EH", "ER", "EY", "IH", "IX", "IY", "OW", "OY", "UH", "UW", "UX"]
    
    word_phones = []
    input_text_phones = [pronouncing.phones_for_word(i) for i in input_text.split()]
    
    # Handle OOV words with simple fallback
    for phones, word in zip(input_text_phones, input_text.split()):
        if phones:
            word_phones.append(phones[0] if isinstance(phones[0], str) else phones[0])
        else:
            # Simple fallback for OOV words - use the word itself
            print(f"Warning: '{word}' not in pronunciation dictionary, using simple phoneme approximation")
            word_phones.append(word.upper())
    
    # Add lexical stress variants to vowels
    word_phones = [re.sub('|'.join(vowel_phones), lambda x: str(x.group(0)) + '[0|1|2]', re.sub(r'\\d+', '', i)) for i in word_phones]
    
    adversarial_phrases = []
    for phones, word in zip(word_phones, input_text.split()):
        query_exps = []
        phones_list = phones.split()
        adversarial_words = []
        if len(phones_list) <= 2:
            query_exps.append(" ".join(phones_list))
        else:
            query_exps.extend(phoneme_replacement(phones_list, max_replace=max(0, len(phones_list)-2), replace_char="(.){1,3}"))
        
        for query in query_exps:
            try:
                matches = pronouncing.search(query)
                matches_phones = [pronouncing.phones_for_word(i)[0] for i in matches if pronouncing.phones_for_word(i)]
                allowed_matches = [i for i, j in zip(matches, matches_phones) if j != phones]
                adversarial_words.extend([i for i in allowed_matches if word.lower() != i])
            except Exception:
                pass
        
        if adversarial_words:
            adversarial_phrases.append(adversarial_words)
    
    # Build combinations
    adversarial_texts = []
    if not adversarial_phrases:
        return adversarial_texts
    
    for i in range(N):
        txts = []
        for j, k in zip(adversarial_phrases, input_text.split()):
            if np.random.random() > (1 - include_input_words):
                txts.append(k)
            else:
                txts.append(np.random.choice(j))
        
        if include_partial_phrase and len(input_text.split()) > 1 and np.random.random() <= include_partial_phrase:
            n_words = np.random.randint(1, len(input_text.split()) + 1)
            adversarial_texts.append(" ".join(np.random.choice(txts, size=n_words, replace=False)))
        else:
            adversarial_texts.append(" ".join(txts))
    
    # Remove exact matches
    adversarial_texts = [i for i in adversarial_texts if i != input_text]
    return adversarial_texts

# Generate adversarial texts
adversarial_texts = []
for word in TARGET_WORD.split():
    adversarial_texts.extend(generate_adversarial_texts(
        input_text=word,
        N=N_SAMPLES_TRAIN // max(1, len(TARGET_WORD.split())),
        include_partial_phrase=0.5,
        include_input_words=0.2
    ))

print(f"Generated {len(adversarial_texts)} adversarial phrases")
print(f"Examples: {adversarial_texts[:5]}")

In [ ]:
# Generate negative training samples
print(f"\nGenerating {N_SAMPLES_TRAIN} negative training samples...")

length_scales = [0.8, 0.9, 1.0, 1.1, 1.2]
noise_scales = [0.5, 0.667, 0.8]
noise_w_scales = [0.6, 0.8, 1.0]

generated = 0
for i in tqdm(range(N_SAMPLES_TRAIN), desc="Negative train"):
    output_path = NEGATIVE_TRAIN_DIR / f"sample_{i:05d}.wav"
    if output_path.exists():
        generated += 1
        continue
    
    text = random.choice(adversarial_texts)
    speaker_id = random.randint(0, num_speakers - 1)
    
    try:
        generate_sample(
            text, output_path,
            speaker_id=speaker_id,
            length_scale=random.choice(length_scales),
            noise_scale=random.choice(noise_scales),
            noise_w_scale=random.choice(noise_w_scales),
        )
        generated += 1
    except Exception as e:
        pass  # Some adversarial texts may fail, that's ok

print(f"Generated {generated} negative training samples")

In [ ]:
# Generate negative test samples
print(f"\nGenerating {N_SAMPLES_TEST} negative test samples...")

generated = 0
for i in tqdm(range(N_SAMPLES_TEST), desc="Negative test"):
    output_path = NEGATIVE_TEST_DIR / f"sample_{i:05d}.wav"
    if output_path.exists():
        generated += 1
        continue
    
    text = random.choice(adversarial_texts)
    speaker_id = random.randint(0, num_speakers - 1)
    
    try:
        generate_sample(
            text, output_path,
            speaker_id=speaker_id,
            length_scale=random.choice(length_scales),
            noise_scale=random.choice(noise_scales),
            noise_w_scale=random.choice(noise_w_scales),
        )
        generated += 1
    except Exception as e:
        pass

print(f"Generated {generated} negative test samples")

## 6. Data Augmentation (Windows-Compatible)

Apply audio augmentations to simulate real-world conditions:
- Room reverb (RIR convolution)
- Background noise mixing
- Volume variation
- Pitch shifting
- Speed variation

In [ ]:
def load_audio(path: Path, target_sr: int = 16000) -> torch.Tensor:
    """Load audio file and resample to target sample rate."""
    waveform, sr = torchaudio.load(str(path))
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(sr, target_sr)
        waveform = resampler(waveform)
    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    return waveform.squeeze(0)


def apply_rir(waveform: torch.Tensor, rir: torch.Tensor) -> torch.Tensor:
    """Apply room impulse response via convolution."""
    # Normalize RIR
    rir = rir / (rir.abs().max() + 1e-8)
    # Convolve
    result = torch.nn.functional.conv1d(
        waveform.unsqueeze(0).unsqueeze(0),
        rir.unsqueeze(0).unsqueeze(0),
        padding=rir.shape[0] // 2
    ).squeeze()
    # Trim to original length
    return result[:waveform.shape[0]]


def mix_audio(foreground: torch.Tensor, background: torch.Tensor, snr_db: float) -> torch.Tensor:
    """Mix foreground with background at specified SNR."""
    # Ensure background is long enough
    while background.shape[0] < foreground.shape[0]:
        background = torch.cat([background, background])
    
    # Random start position in background
    max_start = background.shape[0] - foreground.shape[0]
    if max_start > 0:
        start = random.randint(0, max_start)
        background = background[start:start + foreground.shape[0]]
    else:
        background = background[:foreground.shape[0]]
    
    # Calculate scaling for desired SNR
    fg_rms = foreground.pow(2).mean().sqrt() + 1e-8
    bg_rms = background.pow(2).mean().sqrt() + 1e-8
    snr_linear = 10 ** (snr_db / 20)
    scale = fg_rms / (bg_rms * snr_linear)
    
    mixed = foreground + background * scale
    # Normalize to prevent clipping
    max_val = mixed.abs().max()
    if max_val > 1.0:
        mixed = mixed / max_val
    
    return mixed


def create_fixed_length_clip(waveform: torch.Tensor, target_length: int, 
                              end_jitter_ms: int = 200, sr: int = 16000) -> torch.Tensor:
    """Pad/truncate clip to fixed length, with the clip near the end."""
    result = torch.zeros(target_length)
    
    # Random jitter for end position
    end_jitter_samples = int(random.uniform(0, end_jitter_ms / 1000) * sr)
    
    if waveform.shape[0] >= target_length:
        # Truncate - take from random position
        if random.random() > 0.5:
            result = waveform[:target_length]
        else:
            result = waveform[-target_length:]
    else:
        # Pad - position clip near end with jitter
        start = max(0, target_length - waveform.shape[0] - end_jitter_samples)
        end = start + waveform.shape[0]
        result[start:end] = waveform
    
    return result


def augment_clip(waveform: torch.Tensor, 
                 target_length: int,
                 rir_paths: list,
                 noise_paths: list,
                 sr: int = 16000,
                 rir_prob: float = 0.5,
                 noise_prob: float = 0.75,
                 pitch_shift_prob: float = 0.25,
                 speed_prob: float = 0.25) -> torch.Tensor:
    """Apply augmentations to a single clip."""
    
    # Speed perturbation (before padding)
    if random.random() < speed_prob:
        speed_factor = random.uniform(0.9, 1.1)
        effects = [["speed", str(speed_factor)], ["rate", str(sr)]]
        waveform_np = waveform.numpy()
        waveform, _ = torchaudio.sox_effects.apply_effects_tensor(
            torch.from_numpy(waveform_np).unsqueeze(0), sr, effects
        )
        waveform = waveform.squeeze(0)
    
    # Pitch shift
    if random.random() < pitch_shift_prob:
        n_steps = random.uniform(-3, 3)
        waveform = torchaudio.functional.pitch_shift(waveform.unsqueeze(0), sr, n_steps).squeeze(0)
    
    # Create fixed-length clip
    waveform = create_fixed_length_clip(waveform, target_length, sr=sr)
    
    # Apply RIR (reverb)
    if rir_paths and random.random() < rir_prob:
        try:
            rir = load_audio(random.choice(rir_paths), sr)
            waveform = apply_rir(waveform, rir)
        except Exception:
            pass  # Skip if RIR loading fails
    
    # Mix with background noise
    if noise_paths and random.random() < noise_prob:
        try:
            noise = load_audio(random.choice(noise_paths), sr)
            snr_db = random.uniform(-5, 20)  # Wide range of SNRs
            waveform = mix_audio(waveform, noise, snr_db)
        except Exception:
            pass  # Skip if noise loading fails
    
    # Volume augmentation
    volume = random.uniform(0.1, 1.0)
    waveform = waveform * (volume / (waveform.abs().max() + 1e-8))
    
    return waveform


def augment_clips_generator(clip_paths: list,
                            target_length: int,
                            rir_paths: list,
                            noise_paths: list,
                            batch_size: int = 64,
                            augmentation_rounds: int = 1,
                            sr: int = 16000):
    """Generator that yields batches of augmented clips as 16-bit PCM."""
    
    # Expand clip list for augmentation rounds
    all_clips = clip_paths * augmentation_rounds
    random.shuffle(all_clips)
    
    for i in range(0, len(all_clips), batch_size):
        batch_paths = all_clips[i:i + batch_size]
        batch = []
        
        for clip_path in batch_paths:
            try:
                waveform = load_audio(clip_path, sr)
                augmented = augment_clip(
                    waveform, target_length, rir_paths, noise_paths, sr
                )
                # Convert to 16-bit PCM
                pcm = (augmented.numpy() * 32767).astype(np.int16)
                batch.append(pcm)
            except Exception as e:
                continue  # Skip failed clips
        
        if batch:
            yield np.stack(batch)

print("Augmentation functions defined")

In [ ]:
# Determine clip length from samples
positive_clips = list(POSITIVE_TEST_DIR.glob("*.wav"))
durations = []
for clip in positive_clips[:50]:
    waveform = load_audio(clip)
    durations.append(waveform.shape[0])

median_duration = int(np.median(durations))
# Add 750ms buffer and round to nearest 1000
TOTAL_LENGTH = int(np.ceil((median_duration + 12000) / 1000) * 1000)
# Minimum 2 seconds
TOTAL_LENGTH = max(TOTAL_LENGTH, 32000)
# Clamp near 32000 to exactly 32000
if abs(TOTAL_LENGTH - 32000) <= 4000:
    TOTAL_LENGTH = 32000

print(f"Median clip duration: {median_duration} samples ({median_duration/16000:.2f}s)")
print(f"Target augmented length: {TOTAL_LENGTH} samples ({TOTAL_LENGTH/16000:.2f}s)")

In [ ]:
# Test augmentation on a single clip
test_clip = list(POSITIVE_TRAIN_DIR.glob("*.wav"))[0]
waveform = load_audio(test_clip)
augmented = augment_clip(waveform, TOTAL_LENGTH, rir_paths, noise_paths)

print(f"Original shape: {waveform.shape}")
print(f"Augmented shape: {augmented.shape}")

# Save test augmentation
test_aug_path = SCRIPT_DIR / "test_augmented.wav"
torchaudio.save(str(test_aug_path), augmented.unsqueeze(0), 16000)
display(Audio(str(test_aug_path), autoplay=False))
print(f"\nTest augmented audio saved to: {test_aug_path}")

## 7. Compute Features

Convert augmented audio to openWakeWord features and save as numpy arrays.

In [ ]:
# Import openwakeword feature computation
from openwakeword.utils import AudioFeatures

# Initialize feature extractor
device = "gpu" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

feature_extractor = AudioFeatures(device=device)

In [ ]:
def compute_features_from_augmented_generator(generator, n_total: int, clip_duration: int,
                                               output_file: str, feature_extractor):
    """Compute openWakeWord features from augmented audio generator."""
    
    # Calculate output shape
    # Features are computed every 1280 samples (80ms), starting after 12400 samples (775ms)
    n_features_per_clip = (clip_duration - 12400) // 1280 + 1
    feature_dim = 96  # openWakeWord embedding dimension
    
    # Create memory-mapped output file
    output_shape = (n_total, n_features_per_clip, feature_dim)
    mmap_file = open_memmap(output_file, mode='w+', dtype=np.float32, shape=output_shape)
    
    idx = 0
    pbar = tqdm(total=n_total, desc="Computing features")
    
    for batch in generator:
        # batch is (batch_size, samples) as int16
        for clip in batch:
            if idx >= n_total:
                break
            
            # Compute features for single clip
            features = feature_extractor.embed_clips(clip[np.newaxis, :], batch_size=1)
            
            if features.shape[1] == n_features_per_clip:
                mmap_file[idx] = features[0]
                idx += 1
                pbar.update(1)
        
        mmap_file.flush()
        
        if idx >= n_total:
            break
    
    pbar.close()
    print(f"Saved {idx} feature vectors to {output_file}")
    return idx

In [ ]:
# Check if features already exist
positive_train_features = MODEL_DIR / "positive_features_train.npy"
positive_test_features = MODEL_DIR / "positive_features_test.npy"
negative_train_features = MODEL_DIR / "negative_features_train.npy"
negative_test_features = MODEL_DIR / "negative_features_test.npy"

RECOMPUTE_FEATURES = False  # Set to True to recompute even if files exist

all_exist = all(f.exists() for f in [
    positive_train_features, positive_test_features,
    negative_train_features, negative_test_features
])

if all_exist and not RECOMPUTE_FEATURES:
    print("Feature files already exist. Set RECOMPUTE_FEATURES=True to regenerate.")
else:
    print("Will compute features for all samples...")

In [ ]:
if not positive_train_features.exists() or RECOMPUTE_FEATURES:
    print("\n" + "="*50)
    print("Computing positive training features...")
    print("="*50)
    
    clip_paths = list(POSITIVE_TRAIN_DIR.glob("*.wav"))
    n_total = len(clip_paths) * AUGMENTATION_ROUNDS
    
    generator = augment_clips_generator(
        clip_paths, TOTAL_LENGTH, rir_paths, noise_paths,
        batch_size=64, augmentation_rounds=AUGMENTATION_ROUNDS
    )
    
    compute_features_from_augmented_generator(
        generator, n_total, TOTAL_LENGTH,
        str(positive_train_features), feature_extractor
    )

In [ ]:
if not negative_train_features.exists() or RECOMPUTE_FEATURES:
    print("\n" + "="*50)
    print("Computing negative training features...")
    print("="*50)
    
    clip_paths = list(NEGATIVE_TRAIN_DIR.glob("*.wav"))
    n_total = len(clip_paths) * AUGMENTATION_ROUNDS
    
    generator = augment_clips_generator(
        clip_paths, TOTAL_LENGTH, rir_paths, noise_paths,
        batch_size=64, augmentation_rounds=AUGMENTATION_ROUNDS
    )
    
    compute_features_from_augmented_generator(
        generator, n_total, TOTAL_LENGTH,
        str(negative_train_features), feature_extractor
    )

In [ ]:
if not positive_test_features.exists() or RECOMPUTE_FEATURES:
    print("\n" + "="*50)
    print("Computing positive test features...")
    print("="*50)
    
    clip_paths = list(POSITIVE_TEST_DIR.glob("*.wav"))
    n_total = len(clip_paths) * AUGMENTATION_ROUNDS
    
    generator = augment_clips_generator(
        clip_paths, TOTAL_LENGTH, rir_paths, noise_paths,
        batch_size=64, augmentation_rounds=AUGMENTATION_ROUNDS
    )
    
    compute_features_from_augmented_generator(
        generator, n_total, TOTAL_LENGTH,
        str(positive_test_features), feature_extractor
    )

In [ ]:
if not negative_test_features.exists() or RECOMPUTE_FEATURES:
    print("\n" + "="*50)
    print("Computing negative test features...")
    print("="*50)
    
    clip_paths = list(NEGATIVE_TEST_DIR.glob("*.wav"))
    n_total = len(clip_paths) * AUGMENTATION_ROUNDS
    
    generator = augment_clips_generator(
        clip_paths, TOTAL_LENGTH, rir_paths, noise_paths,
        batch_size=64, augmentation_rounds=AUGMENTATION_ROUNDS
    )
    
    compute_features_from_augmented_generator(
        generator, n_total, TOTAL_LENGTH,
        str(negative_test_features), feature_extractor
    )

In [ ]:
# Verify feature files
print("\nFeature files:")
for f in [positive_train_features, positive_test_features, negative_train_features, negative_test_features]:
    if f.exists():
        data = np.load(str(f), mmap_mode='r')
        print(f"  {f.name}: shape={data.shape}")
    else:
        print(f"  {f.name}: NOT FOUND")

## 8. Download Training Data

Download pre-computed negative examples for training (diverse speech that's NOT your wake word).

In [ ]:
SKIP_LARGE_DOWNLOAD = False  # Set to True to skip the 16GB download

In [ ]:
# Download validation features (small, always download)
VAL_PATH = SCRIPT_DIR / "validation_set_features.npy"

if not VAL_PATH.exists():
    print("Downloading validation features...")
    url = "https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/validation_set_features.npy"
    response = requests.get(url, stream=True)
    response.raise_for_status()
    total = int(response.headers.get("content-length", 0))
    with open(VAL_PATH, "wb") as f:
        with tqdm(total=total, unit="B", unit_scale=True, desc="Validation features") as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                pbar.update(len(chunk))
else:
    print("Validation features already downloaded")

In [ ]:
# Download training features (large)
FEATURES_PATH = SCRIPT_DIR / "openwakeword_features_ACAV100M_2000_hrs_16bit.npy"

if SKIP_LARGE_DOWNLOAD:
    print("Skipping large feature download (training quality will be reduced)")
elif not FEATURES_PATH.exists():
    print("Downloading training features (16GB, this will take a while)...")
    url = "https://huggingface.co/datasets/davidscripka/openwakeword_features/resolve/main/openwakeword_features_ACAV100M_2000_hrs_16bit.npy"
    response = requests.get(url, stream=True)
    response.raise_for_status()
    total = int(response.headers.get("content-length", 0))
    with open(FEATURES_PATH, "wb") as f:
        with tqdm(total=total, unit="B", unit_scale=True, desc="Training features") as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                pbar.update(len(chunk))
else:
    print("Training features already downloaded")

## 9. Train the Model

Now we train using openwakeword's training script, but skip the augmentation step since we already computed features.

In [ ]:
# Training parameters
N_STEPS = 10000
FALSE_ACTIVATION_PENALTY = 1500

In [ ]:
import yaml

# Load default config
with open(OWW_DIR / "examples" / "custom_model.yml") as f:
    config = yaml.safe_load(f)

# Modify config
config["target_phrase"] = [TARGET_WORD]
config["model_name"] = MODEL_OUTPUT_NAME
config["output_dir"] = str(OUTPUT_DIR)
config["steps"] = N_STEPS
config["max_negative_weight"] = FALSE_ACTIVATION_PENALTY
config["target_accuracy"] = 0.5
config["target_recall"] = 0.25

# Data paths - use our pre-computed features
config["false_positive_validation_data_path"] = str(VAL_PATH)

# Feature files for negative examples
if FEATURES_PATH.exists():
    config["feature_data_files"] = {"ACAV100M_sample": str(FEATURES_PATH)}
else:
    config["feature_data_files"] = {}

# These won't be used since we skip generation/augmentation
config["background_paths"] = []
config["rir_paths"] = []
config["n_samples"] = N_SAMPLES_TRAIN
config["n_samples_val"] = N_SAMPLES_TEST

# Save config
CONFIG_YAML_PATH = SCRIPT_DIR / "my_model.yaml"
with open(CONFIG_YAML_PATH, "w") as f:
    yaml.dump(config, f)

print(f"Config saved to: {CONFIG_YAML_PATH}")
print(f"\nTraining configuration:")
print(f"  Target word: {TARGET_WORD}")
print(f"  Steps: {N_STEPS}")
print(f"  False activation penalty: {FALSE_ACTIVATION_PENALTY}")
print(f"  Output: {OUTPUT_DIR}")

In [ ]:
# Run training directly using openwakeword's Model class
# This avoids the subprocess call and allows us to use our pre-computed features

from openwakeword.train import Model
from openwakeword.data import mmap_batch_generator

# Get input shape from features
input_shape = np.load(str(positive_test_features), mmap_mode='r').shape[1:]
print(f"Input shape: {input_shape}")

# Create model
oww_model = Model(
    n_classes=1,
    input_shape=input_shape,
    model_type=config.get("model_type", "dnn"),
    layer_dim=config.get("layer_size", 128),
    seconds_per_example=1280 * input_shape[0] / 16000
)

print("Model created:")
oww_model.summary()

In [ ]:
# Create data transform function for batch generation
def transform_func(x, n=input_shape[0]):
    """Ensure data has correct shape for model."""
    if n > x.shape[1] or n < x.shape[1]:
        x = np.vstack(x)
        new_batch = np.array([x[i:i+n, :] for i in range(0, x.shape[0]-n, n)])
    else:
        return x
    return new_batch

# Setup data files
feature_data_files = dict(config.get("feature_data_files", {}))
feature_data_files["positive"] = str(positive_train_features)
feature_data_files["adversarial_negative"] = str(negative_train_features)

# Data transforms
data_transforms = {key: transform_func for key in feature_data_files.keys()}

# Label transforms
label_transforms = {}
for key in feature_data_files.keys():
    if key == "positive":
        label_transforms[key] = lambda x: [1 for i in x]
    else:
        label_transforms[key] = lambda x: [0 for i in x]

print(f"Feature data files: {list(feature_data_files.keys())}")

In [ ]:
# Create data loaders
batch_generator = mmap_batch_generator(
    feature_data_files,
    n_per_class=config.get("batch_n_per_class", {}),
    data_transform_funcs=data_transforms,
    label_transform_funcs=label_transforms
)

class IterDataset(torch.utils.data.IterableDataset):
    def __init__(self, generator):
        self.generator = generator
    def __iter__(self):
        return self.generator

n_cpus = max(1, (os.cpu_count() or 1) // 2)
X_train = torch.utils.data.DataLoader(
    IterDataset(batch_generator),
    batch_size=None,
    num_workers=n_cpus,
    prefetch_factor=16
)

# Validation data for false positives
X_val_fp = np.load(str(VAL_PATH))
X_val_fp = np.array([X_val_fp[i:i+input_shape[0]] for i in range(0, X_val_fp.shape[0]-input_shape[0], 1)])
X_val_fp_labels = np.zeros(X_val_fp.shape[0]).astype(np.float32)
X_val_fp_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(torch.from_numpy(X_val_fp), torch.from_numpy(X_val_fp_labels)),
    batch_size=len(X_val_fp_labels)
)

# Validation data for accuracy/recall
X_val_pos = np.load(str(positive_test_features))
X_val_neg = np.load(str(negative_test_features))
labels = np.hstack((np.ones(X_val_pos.shape[0]), np.zeros(X_val_neg.shape[0]))).astype(np.float32)

X_val_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.from_numpy(np.vstack((X_val_pos, X_val_neg))),
        torch.from_numpy(labels)
    ),
    batch_size=len(labels)
)

print(f"Training data: {len(feature_data_files)} sources")
print(f"Validation FP data: {X_val_fp.shape}")
print(f"Validation pos/neg data: {X_val_pos.shape[0]} pos, {X_val_neg.shape[0]} neg")

In [ ]:
print("="*50)
print("Starting training...")
print("="*50)

best_model = oww_model.auto_train(
    X_train=X_train,
    X_val=X_val_loader,
    false_positive_val_data=X_val_fp_loader,
    steps=N_STEPS,
    max_negative_weight=FALSE_ACTIVATION_PENALTY,
    target_fp_per_hour=config.get("target_false_positives_per_hour", 0.5),
)

print("\nTraining complete!")

In [ ]:
# Export the trained model
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

oww_model.export_model(
    model=best_model,
    model_name=MODEL_OUTPUT_NAME,
    output_dir=str(OUTPUT_DIR)
)

print(f"\nModel exported to: {OUTPUT_DIR}")

## 10. Done!

Your trained model is in the `my_custom_model` folder.

In [ ]:
print(f"\nTraining complete! Model files:")
if OUTPUT_DIR.exists():
    for f in OUTPUT_DIR.glob("*"):
        if f.is_file():
            size_mb = f.stat().st_size / (1024 * 1024)
            print(f"  {f.name} ({size_mb:.2f} MB)")
else:
    print(f"  Output directory not found: {OUTPUT_DIR}")

In [ ]:
# Optional: Copy to wakewords folder
import shutil

WAKEWORD_DIR = SCRIPT_DIR.parent / "wakewords" / TARGET_WORD.lower().replace(" ", "_")
WAKEWORD_DIR.mkdir(parents=True, exist_ok=True)

for ext in [".onnx", ".tflite"]:
    src = OUTPUT_DIR / f"{MODEL_OUTPUT_NAME}{ext}"
    if src.exists():
        dst = WAKEWORD_DIR / f"{MODEL_OUTPUT_NAME}{ext}"
        shutil.copy(src, dst)
        print(f"Copied {src.name} to {dst}")

In [ ]:
# Test the model
import openwakeword
from openwakeword.model import Model as OWWModel

# Load trained model
model_path = OUTPUT_DIR / f"{MODEL_OUTPUT_NAME}.onnx"
if model_path.exists():
    oww = OWWModel(wakeword_models=[str(model_path)])
    
    # Test on a positive sample
    test_clip = list(POSITIVE_TEST_DIR.glob("*.wav"))[0]
    audio, sr = torchaudio.load(str(test_clip))
    audio = audio.squeeze().numpy()
    
    # Convert to int16
    audio_int16 = (audio * 32767).astype(np.int16)
    
    # Run prediction
    prediction = oww.predict(audio_int16)
    print(f"Test prediction on positive sample: {prediction}")
else:
    print(f"Model not found at {model_path}")